In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path

from src.processing.splitting import time_based_split
from src.processing.scaling import scale_features
from src.processing.sequence_creator import create_sequences
from src.training.lstm_training import train_lstm
from config import config


df_btc = pd.read_csv(
    config.DATA_DIR / "processed" / "crypto_prices" / "btc.csv",
    parse_dates=["date"],
    index_col="date",
)
df_eth = pd.read_csv(
    config.DATA_DIR / "processed" / "crypto_prices" / "eth.csv",
    parse_dates=["date"],
    index_col="date",
)

## Split


In [2]:
X_train_btc, y_train_btc, X_val_btc, y_val_btc, X_test_btc, y_test_btc = (
    time_based_split(
        df_btc,
        targets=["logPriceChange"],
        test_months=12,
        val_months=3,
        lags=30,
    )
)

X_train_eth, y_train_eth, X_val_eth, y_val_eth, X_test_eth, y_test_eth = (
    time_based_split(
        df_eth,
        targets=["logPriceChange"],
        test_months=12,
        val_months=3,
        lags=30,
    )
)

/Users/matti/coding/cryptoCapstone/src/processing/splitting.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[target] = train[target].shift(-1)
/Users/matti/coding/cryptoCapstone/src/processing/splitting.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val[target] = val[target].shift(-1)
/Users/matti/coding/cryptoCapstone/src/processing/splitting.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [3]:
print(y_test_btc.tail())
print(y_test_btc.head())

            logPriceChange
date                      
2022-12-15       -0.042190
2022-12-16        0.008828
2022-12-17       -0.002212
2022-12-18       -0.019176
2022-12-19        0.027989
            logPriceChange
date                      
2021-11-21       -0.042455
2021-11-22        0.022481
2021-11-23       -0.022639
2021-11-24        0.017512
2021-11-25       -0.066874


## Scale


In [4]:
X_train_scaled_btc, X_val_scaled_btc, X_test_scaled_btc, scaler_btc = scale_features(
    X_train_btc, X_val_btc, X_test_btc
)

X_train_scaled_eth, X_val_scaled_eth, X_test_scaled_eth, scaler_eth = scale_features(
    X_train_eth, X_val_eth, X_test_eth
)

## Sequence Creation


In [5]:
X_train_seq_btc, y_train_seq_btc = create_sequences(
    X_train_scaled_btc, y_train_btc, seq_length=30
)
X_val_seq_btc, y_val_seq_btc = create_sequences(
    X_val_scaled_btc, y_val_btc, seq_length=30
)
X_test_seq_btc, y_test_seq_btc = create_sequences(
    X_test_scaled_btc, y_test_btc, seq_length=30
)

print(f"X_train_seq_btc shape: {X_train_seq_btc.shape}")
print(f"y_train_seq_btc shape: {y_train_seq_btc.shape}")

X_train_seq_eth, y_train_seq_eth = create_sequences(
    X_train_scaled_eth, y_train_eth, seq_length=30
)
X_val_seq_eth, y_val_seq_eth = create_sequences(
    X_val_scaled_eth, y_val_eth, seq_length=30
)
X_test_seq_eth, y_test_seq_eth = create_sequences(
    X_test_scaled_eth, y_test_eth, seq_length=30
)

X_train_seq_btc shape: (1662, 30, 72)
y_train_seq_btc shape: (1662, 1)


# BTC


### Model Tuning

In [6]:
from src.training.hyperparameters_tuning import tune_hyperparameters

study_lstm_btc = tune_hyperparameters(
    X_train_seq_btc,
    y_train_seq_btc,
    X_val_seq_btc,
    y_val_seq_btc,
    train_lstm,
    n_trials=50,
)

[I 2025-02-10 16:02:22,776] A new study created in memory with name: no-name-21652274-bc93-4a23-b566-0db2414ed332


Epoch 1/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 4s 87ms/step - loss: 0.1741 - mae: 0.2709 - val_loss: 0.0011 - val_mae: 0.0261
Epoch 2/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 82ms/step - loss: 0.0027 - mae: 0.0364 - val_loss: 0.0011 - val_mae: 0.0260
Epoch 3/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 83ms/step - loss: 0.0021 - mae: 0.0319 - val_loss: 0.0011 - val_mae: 0.0261
Epoch 4/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 86ms/step - loss: 0.0020 - mae: 0.0313 - val_loss: 0.0011 - val_mae: 0.0261
Epoch 5/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 97ms/step - loss: 0.0020 - mae: 0.0312 - val_loss: 0.0011 - val_mae: 0.0261
Epoch 6/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 104ms/step - loss: 0.0020 - mae: 0.0311 - val_loss: 0.0011 - val_mae: 0.0261
Epoch 7/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 102ms/step - loss: 0.0020 - mae: 0.0310 - val_loss: 0.0011 - val_mae: 0.0261
Epoch 8/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 109ms/step - loss: 0.0020 - mae: 0.0311 - val_loss: 0.0011 - val_mae: 0.0261
Epoch 9/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 106ms/step 

[I 2025-02-10 16:03:23,733] Trial 0 finished with value: 0.001104002702049911 and parameters: {'num_lstm_layers': 2, 'lstm_units_1': 256, 'lstm_units_2': 192, 'num_dense_layers': 1, 'dense_units_1': 96, 'dropout_rate': 0.5, 'learning_rate': 0.007122094838712671, 'batch_size': 64}. Best is trial 0 with value: 0.001104002702049911.


Epoch 1/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.2026 - mae: 0.3369 - val_loss: 0.1620 - val_mae: 0.3262
Epoch 2/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.1539 - mae: 0.3014 - val_loss: 0.0480 - val_mae: 0.1900
Epoch 3/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.1849 - mae: 0.3356 - val_loss: 0.0597 - val_mae: 0.2248
Epoch 4/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0534 - mae: 0.1827 - val_loss: 0.0122 - val_mae: 0.0876
Epoch 5/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0288 - mae: 0.1325 - val_loss: 0.0065 - val_mae: 0.0667
Epoch 6/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0233 - mae: 0.1200 - val_loss: 0.0070 - val_mae: 0.0628
Epoch 7/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0183 - mae: 0.1068 - val_loss: 0.0045 - val_mae: 0.0530
Epoch 8/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.0163 - mae: 0.1006 - val_loss: 0.0048 - val_mae: 0.0542
Epoch 9/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - lo

[I 2025-02-10 16:04:18,238] Trial 1 finished with value: 0.0016612129984423518 and parameters: {'num_lstm_layers': 1, 'lstm_units_1': 128, 'num_dense_layers': 0, 'dropout_rate': 0.30000000000000004, 'learning_rate': 0.0015112939600654687, 'batch_size': 64}. Best is trial 0 with value: 0.001104002702049911.


...

Epoch 1/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - loss: 0.0262 - mae: 0.1160 - val_loss: 0.0013 - val_mae: 0.0290
Epoch 2/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0025 - mae: 0.0367 - val_loss: 0.0011 - val_mae: 0.0260
Epoch 3/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0021 - mae: 0.0320 - val_loss: 0.0011 - val_mae: 0.0260
Epoch 4/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 0.0021 - mae: 0.0315 - val_loss: 0.0011 - val_mae: 0.0261
Epoch 5/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - loss: 0.0020 - mae: 0.0312 - val_loss: 0.0011 - val_mae: 0.0261
Epoch 6/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 0.0020 - mae: 0.0308 - val_loss: 0.0011 - val_mae: 0.0260
Epoch 7/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 0.0020 - mae: 0.0306 - val_loss: 0.0011 - val_mae: 0.0261
Epoch 8/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 0.0020 - mae: 0.0306 - val_loss: 0.0011 - val_mae: 0.0260
Epoch 9/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - lo

[I 2025-02-10 18:10:38,660] Trial 48 finished with value: 0.0009435547981411219 and parameters: {'num_lstm_layers': 2, 'lstm_units_1': 192, 'lstm_units_2': 32, 'num_dense_layers': 3, 'dense_units_1': 160, 'dense_units_2': 224, 'dense_units_3': 256, 'dropout_rate': 0.4, 'learning_rate': 0.002648080796009569, 'batch_size': 64}. Best is trial 25 with value: 0.0008958670659922063.


Epoch 1/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 52ms/step - loss: 0.0154 - mae: 0.0924 - val_loss: 0.0012 - val_mae: 0.0275
Epoch 2/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0026 - mae: 0.0364 - val_loss: 0.0012 - val_mae: 0.0270
Epoch 3/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0021 - mae: 0.0320 - val_loss: 0.0011 - val_mae: 0.0263
Epoch 4/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 0.0020 - mae: 0.0312 - val_loss: 0.0011 - val_mae: 0.0262
Epoch 5/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step - loss: 0.0020 - mae: 0.0308 - val_loss: 0.0011 - val_mae: 0.0262
Epoch 6/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step - loss: 0.0020 - mae: 0.0308 - val_loss: 0.0011 - val_mae: 0.0262
Epoch 7/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - loss: 0.0020 - mae: 0.0307 - val_loss: 0.0011 - val_mae: 0.0261
Epoch 8/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 64ms/step - loss: 0.0020 - mae: 0.0308 - val_loss: 0.0011 - val_mae: 0.0261
Epoch 9/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - lo

[I 2025-02-10 18:12:40,331] Trial 49 finished with value: 0.0009430181235074997 and parameters: {'num_lstm_layers': 2, 'lstm_units_1': 192, 'lstm_units_2': 32, 'num_dense_layers': 3, 'dense_units_1': 160, 'dense_units_2': 224, 'dense_units_3': 256, 'dropout_rate': 0.4, 'learning_rate': 0.0023520510599806016, 'batch_size': 64}. Best is trial 25 with value: 0.0008958670659922063.


Best trial:
  Value (val_loss): 0.0008958670659922063
  Params:
    num_lstm_layers: 2
    lstm_units_1: 192
    lstm_units_2: 96
    num_dense_layers: 1
    dense_units_1: 128
    dropout_rate: 0.30000000000000004
    learning_rate: 0.00039292722931752385
    batch_size: 16


## Model Training

In [9]:
# best parameters from the Optuna study
best_params_btc = study_lstm_btc.best_params
# best_params_btc = {
#     "num_lstm_layers": 3,
#     "lstm_units_1": 256,
#     "lstm_units_2": 32,
#     "lstm_units_3": 64,
#     "num_dense_layers": 1,
#     "dense_units_1": 224,
#     "dense_units_2": 224,
#     "dense_units_3": 64,
#     "dropout_rate": 0.30000000000000004,
#     "learning_rate": 0.00035070707045515984,
#     "batch_size": 64,
# }


# Format the parameters correctly
formatted_params = {
    "lstm_units": [
        best_params_btc[f"lstm_units_{i+1}"] for i in range(best_params_btc["num_lstm_layers"])
    ],
    "dense_units": [
        best_params_btc[f"dense_units_{i+1}"]
        for i in range(best_params_btc["num_dense_layers"])
    ],
    "dropout_rate": best_params_btc["dropout_rate"],
    "learning_rate": best_params_btc["learning_rate"],
    "batch_size": best_params_btc["batch_size"],
    "epochs": 300,
}

# Train the model with the formatted parameters
model_btc, history_btc = train_lstm(
    X_train_seq_btc,
    y_train_seq_btc,
    X_val_seq_btc,
    y_val_seq_btc,
    params=formatted_params,
)

Epoch 1/300
104/104 ━━━━━━━━━━━━━━━━━━━━ 5s 35ms/step - loss: 0.0265 - mae: 0.1253 - val_loss: 0.0046 - val_mae: 0.0557
Epoch 2/300
104/104 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - loss: 0.0097 - mae: 0.0760 - val_loss: 0.0018 - val_mae: 0.0339
Epoch 3/300
104/104 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - loss: 0.0057 - mae: 0.0583 - val_loss: 0.0016 - val_mae: 0.0340
Epoch 4/300
104/104 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - loss: 0.0042 - mae: 0.0488 - val_loss: 0.0028 - val_mae: 0.0442
Epoch 5/300
104/104 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - loss: 0.0033 - mae: 0.0429 - val_loss: 0.0012 - val_mae: 0.0273
Epoch 6/300
104/104 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - loss: 0.0027 - mae: 0.0392 - val_loss: 0.0012 - val_mae: 0.0282
Epoch 7/300
104/104 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - loss: 0.0024 - mae: 0.0360 - val_loss: 0.0012 - val_mae: 0.0272
Epoch 8/300
104/104 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - loss: 0.0024 - mae: 0.0358 - val_loss: 0.0012 - val_mae: 0.0274
Epoch 9/300
104/104 ━━━━━━━━━━━━━━━━━━━━

In [11]:
# save weights
model_btc.save_weights(
    config.DATA_DIR
    / "weights"
    / "hloc_only"
    / "log_price_change_weights_btc.weights.h5"
)

#  generate predictions
y_pred_btc = model_btc.predict(X_test_seq_btc).flatten()

# create new df and save it to csv
predictions_btc = pd.DataFrame(
    {
        "date": y_test_btc.index[30:],
        "predictedLogPriceChange": y_pred_btc,
    }
)


predictions_btc.to_csv(
    config.DATA_DIR
    / "predictions"
    / "hloc_only"
    / "log_price_change_predictions_btc.csv",
    index=False,
)

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


# ETH

### Model Tuning

In [12]:
study_lstm_eth = tune_hyperparameters(
    X_train_seq_eth,
    y_train_seq_eth,
    X_val_seq_eth,
    y_val_seq_eth,
    train_lstm,
    n_trials=50,
)

[I 2025-02-10 18:40:49,182] A new study created in memory with name: no-name-2fd03665-889c-4e8f-acac-a7277b9c0bea


Epoch 1/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 4s 75ms/step - loss: 0.0090 - mae: 0.0731 - val_loss: 0.0019 - val_mae: 0.0340
Epoch 2/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 64ms/step - loss: 0.0055 - mae: 0.0549 - val_loss: 0.0019 - val_mae: 0.0331
Epoch 3/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - loss: 0.0049 - mae: 0.0501 - val_loss: 0.0020 - val_mae: 0.0335
Epoch 4/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - loss: 0.0048 - mae: 0.0493 - val_loss: 0.0017 - val_mae: 0.0326
Epoch 5/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - loss: 0.0050 - mae: 0.0492 - val_loss: 0.0020 - val_mae: 0.0361
Epoch 6/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 72ms/step - loss: 0.0046 - mae: 0.0479 - val_loss: 0.0016 - val_mae: 0.0310
Epoch 7/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step - loss: 0.0043 - mae: 0.0460 - val_loss: 0.0018 - val_mae: 0.0335
Epoch 8/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - loss: 0.0045 - mae: 0.0463 - val_loss: 0.0018 - val_mae: 0.0334
Epoch 9/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step - lo

[I 2025-02-10 18:41:38,492] Trial 0 finished with value: 0.001580598414875567 and parameters: {'num_lstm_layers': 3, 'lstm_units_1': 64, 'lstm_units_2': 64, 'lstm_units_3': 256, 'num_dense_layers': 0, 'dropout_rate': 0.2, 'learning_rate': 0.0003636105008375858, 'batch_size': 64}. Best is trial 0 with value: 0.001580598414875567.


Epoch 1/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 5s 67ms/step - loss: 0.0090 - mae: 0.0700 - val_loss: 0.0029 - val_mae: 0.0445
Epoch 2/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - loss: 0.0052 - mae: 0.0515 - val_loss: 0.0018 - val_mae: 0.0330
Epoch 3/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 63ms/step - loss: 0.0049 - mae: 0.0508 - val_loss: 0.0028 - val_mae: 0.0422
Epoch 4/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 4s 67ms/step - loss: 0.0043 - mae: 0.0451 - val_loss: 0.0020 - val_mae: 0.0346
Epoch 5/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 65ms/step - loss: 0.0040 - mae: 0.0438 - val_loss: 0.0017 - val_mae: 0.0323
Epoch 6/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 4s 68ms/step - loss: 0.0042 - mae: 0.0452 - val_loss: 0.0017 - val_mae: 0.0331
Epoch 7/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 67ms/step - loss: 0.0041 - mae: 0.0452 - val_loss: 0.0018 - val_mae: 0.0339
Epoch 8/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - loss: 0.0041 - mae: 0.0442 - val_loss: 0.0025 - val_mae: 0.0387
Epoch 9/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 64ms/step - lo

[I 2025-02-10 18:43:53,652] Trial 1 finished with value: 0.0015946889761835337 and parameters: {'num_lstm_layers': 3, 'lstm_units_1': 96, 'lstm_units_2': 192, 'lstm_units_3': 96, 'num_dense_layers': 2, 'dense_units_1': 160, 'dense_units_2': 224, 'dropout_rate': 0.0, 'learning_rate': 0.0009885377940943617, 'batch_size': 32}. Best is trial 0 with value: 0.001580598414875567.


...

Epoch 1/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.3176 - mae: 0.4180 - val_loss: 0.1169 - val_mae: 0.2992
Epoch 2/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0996 - mae: 0.2476 - val_loss: 0.0295 - val_mae: 0.1428
Epoch 3/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.1019 - mae: 0.2531 - val_loss: 0.0396 - val_mae: 0.1777
Epoch 4/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0330 - mae: 0.1440 - val_loss: 0.0361 - val_mae: 0.1564
Epoch 5/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.0232 - mae: 0.1175 - val_loss: 0.0048 - val_mae: 0.0523
Epoch 6/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0149 - mae: 0.0963 - val_loss: 0.0073 - val_mae: 0.0594
Epoch 7/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.0131 - mae: 0.0867 - val_loss: 0.0057 - val_mae: 0.0618
Epoch 8/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0115 - mae: 0.0814 - val_loss: 0.0056 - val_mae: 0.0515
Epoch 9/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - lo

[I 2025-02-10 19:59:22,440] Trial 48 finished with value: 0.0017887187423184514 and parameters: {'num_lstm_layers': 1, 'lstm_units_1': 128, 'num_dense_layers': 0, 'dropout_rate': 0.4, 'learning_rate': 0.002685637668353882, 'batch_size': 64}. Best is trial 43 with value: 0.0012989953393116593.


Epoch 1/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - loss: 0.1628 - mae: 0.3031 - val_loss: 0.0348 - val_mae: 0.1583
Epoch 2/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0664 - mae: 0.1979 - val_loss: 0.0111 - val_mae: 0.0808
Epoch 3/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0354 - mae: 0.1456 - val_loss: 0.0045 - val_mae: 0.0532
Epoch 4/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0215 - mae: 0.1110 - val_loss: 0.0026 - val_mae: 0.0385
Epoch 5/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0103 - mae: 0.0770 - val_loss: 0.0030 - val_mae: 0.0462
Epoch 6/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.0088 - mae: 0.0697 - val_loss: 0.0019 - val_mae: 0.0345
Epoch 7/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.0067 - mae: 0.0605 - val_loss: 0.0017 - val_mae: 0.0337
Epoch 8/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0060 - mae: 0.0565 - val_loss: 0.0016 - val_mae: 0.0322
Epoch 9/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - lo

[I 2025-02-10 20:01:12,462] Trial 49 finished with value: 0.0013350793160498142 and parameters: {'num_lstm_layers': 1, 'lstm_units_1': 224, 'num_dense_layers': 1, 'dense_units_1': 160, 'dropout_rate': 0.5, 'learning_rate': 0.001206556001512851, 'batch_size': 64}. Best is trial 43 with value: 0.0012989953393116593.


Best trial:
  Value (val_loss): 0.0012989953393116593
  Params:
    num_lstm_layers: 1
    lstm_units_1: 160
    num_dense_layers: 1
    dense_units_1: 160
    dropout_rate: 0.2
    learning_rate: 0.00638524743226478
    batch_size: 64


## Model Training

In [ ]:
# best parameters from the Optuna study
best_params_eth = study_lstm_eth.best_params_eth
# best_params_eth = {
#     "num_lstm_layers": 2,
#     "lstm_units_1": 160,
#     "lstm_units_2": 32,
#     "lstm_units_3": 32,
#     "num_dense_layers": 1,
#     "dense_units_1": 96,
#     "dense_units_2": 64,
#     "dense_units_3": 192,
#     "dropout_rate": 0.2,
#     "learning_rate": 0.0005577190382479473,
#     "batch_size": 32,
# }


# Format the parameters correctly
formatted_params = {
    "lstm_units": [
        best_params_eth[f"lstm_units_{i+1}"] for i in range(best_params_eth["num_lstm_layers"])
    ],
    "dense_units": [
        best_params_eth[f"dense_units_{i+1}"]
        for i in range(best_params_eth["num_dense_layers"])
    ],
    "dropout_rate": best_params_eth["dropout_rate"],
    "learning_rate": best_params_eth["learning_rate"],
    "batch_size": best_params_eth["batch_size"],
    "epochs": 300,
}

# Train the model with the formatted parameters
model_eth, history_eth = train_lstm(
    X_train_seq_eth,
    y_train_seq_eth,
    X_val_seq_eth,
    y_val_seq_eth,
    params=formatted_params,
)

Epoch 1/300
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - loss: 0.0194 - mae: 0.1052 - val_loss: 0.0025 - val_mae: 0.0384
Epoch 2/300
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - loss: 0.0085 - mae: 0.0691 - val_loss: 0.0031 - val_mae: 0.0460
Epoch 3/300
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - loss: 0.0067 - mae: 0.0609 - val_loss: 0.0022 - val_mae: 0.0372
Epoch 4/300
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - loss: 0.0054 - mae: 0.0528 - val_loss: 0.0015 - val_mae: 0.0300
Epoch 5/300
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0049 - mae: 0.0504 - val_loss: 0.0017 - val_mae: 0.0335
Epoch 6/300
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - loss: 0.0046 - mae: 0.0474 - val_loss: 0.0020 - val_mae: 0.0355
Epoch 7/300
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.0046 - mae: 0.0474 - val_loss: 0.0017 - val_mae: 0.0331
Epoch 8/300
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 0.0046 - mae: 0.0471 - val_loss: 0.0016 - val_mae: 0.0317
Epoch 9/300
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - lo

In [ ]:
# save weights
model_eth.save_weights(
    config.DATA_DIR
    / "weights"
    / "hloc_only"
    / "log_price_change_weights_eth.weights.h5"
)

#  generate predictions
y_pred_eth = model_eth.predict(X_test_seq_eth).flatten()

# create new df and save it to csv
predictions_eth = pd.DataFrame(
    {
        "date": y_test_eth.index[30:],
        "predictedLogPriceChange": y_pred_eth,
    }
)


predictions_eth.to_csv(
    config.DATA_DIR
    / "predictions"
    / "hloc_only"
    / "log_price_change_predictions_eth.csv",
    index=False,
)

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
